![RedTen](http://i.imgur.com/pI7TE3w.png)

## Predicting the SPY's Future Closing Price with a Multi-Model Forecast

Creating many machine learning models to predict future price movements from Redis.

![If only managing my portfolio was this easy...](http://i.imgur.com/2ectZu8.png)

### How?

1. Uses pricing metrics (hlocv)
1. Streamline development and deployment of machine learning forecasts by storing large, pre-trained models living in Redis
1. Custom rolled dataset (takes about 7 hours per 1 ticker)
1. Technical indicators



### Why?

1. Took too long to manually rebuild the dataset, and build + tune new models
1. Improve model accuracy by tracking success (situational/seasonal risks)
1. Wanted simple, consistent delivery of results
1. Service layer for abstracting model implementation
1. Multi-tenant, distributed machine learning cloud
1. Team needed Jupyter integration
1. Data security - so it had to run on-premise and cloud

__Now it takes 30 minutes to build the dataset and 5 minutes to make new predictions__

## Sample SPY Multi-Model Forecast

![Combined HLOCV forecast](http://i.imgur.com/iraWaZV.png)

## Setup the Environment

Load the shared core, methods, and environment before starting processing

In [1]:
from __future__ import print_function
import sys, os, requests, json, datetime

# Load the environment and login the user
from src.common.load_redten_ipython_env import user_token, user_login, csv_file, run_job, core, api_urls, ppj, rt_url, rt_user, rt_pass, rt_email, lg, good, boom, anmt, mark, ppj, uni_key, rest_login_as_user, rest_full_login, wait_for_job_to_finish, wait_on_job, get_job_analysis, get_job_results, get_analysis_manifest, get_job_cache_manifest, build_prediction_results, build_forecast_results, get_job_cache_manifest, search_ml_jobs, show_logs, show_errors, ipyImage, ipyHTML, ipyDisplay, pd, np

## Configure the job

In [2]:
# dataset name is the ticker
ds_name = "SPY"

# Label and description for job
title = str(ds_name) + " Forecast v5 - " + str(uni_key())
desc = "Forecast simulation - " + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# Whats the algorithm model you want to use?
algo_name = "xgb-regressor"

# If your dataset is stored in redis, you can pass in the location
# to the dataset like: <redis endpoint>:<key>
rloc = "" 

# If your dataset is stored in S3, you can pass in the location 
# to the dataset like: <bucket>:<key>
sloc = "" 

# During training what ratio of tests-vs-training do you want to use?
# Trade off smarts vs accuracy...how smart are we going?
test_ratio = 0.1

# Customize dataset samples used during the analysis using json dsl
sample_filter_rules = {}

# What column do you want to predict values?
target_column_name = "FClose"

# What columns can the algorithms use for training and learning?
feature_column_names = [ "FHigh", "FLow", "FOpen", "FClose", "FVolume" ]

# values in the Target Column
target_column_values = [ "GoodBuys", "BadBuys", "Not Finished" ] 

# How many units ahead do you want to forecast?
units_ahead_set = [ 5, 10, 15, 20, 25, 30 ]
units_ahead_type = "Days"

# Prune non-int/float columns as needed: 
ignore_features = [       
                "Ticker",
                "Date",
                "FDate",
                "FPrice",
                "DcsnDate",
                "Decision"
            ]

# Set up the XGB parameter
# https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
train_xgb = {
                "learning_rate" : 0.20, 
                "num_estimators" : 50, 
                "sub_sample" : 0.20, 
                "col_sample_by_tree" : 0.90, 
                "col_sample_by_level" : 1.0, 
                "objective" : "reg:linear",
                "max_depth" : 3,
                "max_delta_step" : 0,
                "min_child_weight" : 1, 
                "reg_alpha" : 0, 
                "reg_lambda" : 1,
                "base_score" : 0.6,
                "gamma" : 0,
                "seed" : 42, 
                "silent" : True
            } 

# Predict new price points during the day
predict_row = {
                "High"   : 250.82,
                "Low"    : 245.54,
                "Open"   : 247.77,
                "Close"  : 246.24,
                "Volume" : 77670266
            }

## Start Forecasting

In [3]:
job_id = None # on success, this will store the actively running job's id
csv_file = ""
post_data = {
            "predict_this_data" : predict_row,
            "title" : title,
            "desc" : desc,
            "ds_name" : ds_name,
            "target_column_name" : target_column_name,
            "feature_column_names" : feature_column_names,
            "ignore_features" : ignore_features,
            "csv_file" : csv_file,
            "rloc" : rloc,
            "sloc" : sloc,
            "algo_name" : algo_name,
            "test_ratio" : test_ratio,
            "target_column_values" : target_column_values,
            "label_column_name" : target_column_name, 
            "prediction_type" : "Forecast",
            "ml_type" : "Playbook-UnitsAhead",
            "train" : train_xgb,
            "tracking_type" : "",
            "units_ahead_set" : units_ahead_set,
            "units_ahead_type" : units_ahead_type,
            "forecast_type" : "ETFPriceForecasting",
            "sample_filters" : sample_filter_rules,
            "predict_units_back" : 90, # how many days back should the final chart go?
            "send_to_email" : [ "jay.p.h.johnson@gmail.com" ] # comma separated list
        }

anmt("Running job: " + str(title))

auth_headers = { 
            "Content-type": "application/json",
            "Authorization" : "JWT " + str(user_token)
        }

job_response = run_job(post_data=post_data, headers=auth_headers)

if job_response["status"] != "valid":
    boom("Forecast job failed with error=" + str(job_response["status"]))
else:
    if "id" not in job_response["data"]:
        boom("Failed to create new forecast job")
    else:
        job_id = job_response["data"]["id"]
        job_status = job_response["data"]["status"]
        lg("Started Forecast job=" + str(job_id) + " with current status=" + str(job_status))
# end of if job was valid or not

Running job: SPY Forecast v5 - 34d6bf175cad4aaf9234aa06e95f4e4
Started Forecast job=558 with current status=requested


## Wait for the job to finish

In [4]:
job_data = {}
job_report = {}

# Should hook this up to a randomized image loader...
ipyDisplay(ipyImage(url="https://media.giphy.com/media/l397998l2DT0ogare/giphy.gif"))

job_res = {}
if job_id == None:
    boom("Failed to start a new job")
else:
    job_res = wait_on_job(job_id)

    if job_res["status"] != "SUCCESS":
        boom("Job=" + str(job_id) + " failed with status=" + str(job_res["status"]) + " err=" + str(job_res["error"]))
    else:
        job_data = job_res["record"]
        anmt("Job Report:")
        lg(ppj(job_data), 5)
# end of waiting

Waiting on job=558 url=https://redten.io/ml/558/
Job=558 is training - Step 3/10
Job=558 is analyzing - Step 5/10
Job=558 is caching - Step 6/10
Job=558 is plotting - Step 7/10
Job=558 is uploading - Step 9/10
Job=558 completed
Job Report:
{
    "job": {
        "algo_name": "xgb-regressor",
        "control_state": "active",
        "created": "2017-05-26 08-02-13",
        "csv_file": "",
        "desc": "Forecast simulation - 2017-05-26 08:02:13",
        "ds_name": "SPY",
        "feature_column_names": [
            "FHigh",
            "FLow",
            "FOpen",
            "FClose",
            "FVolume"
        ],
        "id": 558,
        "ignore_features": [
            "Ticker",
            "Date",
            "FDate",
            "FPrice",
            "DcsnDate",
            "Decision"
        ],
        "images": [
            {
                "author_name": null,
                "desc": null,
                "id": 12122,
                "image": "https://rt-media.s3.a

## Get Forecast Accuracies

In [5]:
job_report = {}
if job_id == None:
    boom("Failed to start a new job")
else:
    # Get the analysis, but do not auto-show the plots
    job_report = get_job_analysis(job_id, show_plots=False)
    if len(job_report) == 0:
        boom("Job=" + str(job_id) + " failed")
    else:
        lg("")
    # if the job failed
# end of get job analysis

# Build the forecast accuracy dictionary from the analysis
# and show the forecast dataframes
acc_results = build_forecast_results(job_report)
for col in acc_results:
    col_node = acc_results[col]
    
    predictions_df = col_node["predictions_df"]
    date_predictions_df = col_node["date_predictions_df"]
    train_predictions_df = col_node["train_predictions_df"]
    
    lg("--------------------------------------------------")
    # for all columns in the accuracy dictionary:  
    # successful predictions above 90%...how's that error rate though?
    if col_node["accuracy"] > 0.90: 
        good("Column=" + str(col) + " accuracy=" + str(col_node["accuracy"]) + " mse=" + str(col_node["mse"]) + " num_predictions=" + str(len(col_node["date_predictions_df"].index)))
    # successful predictions between 90% and 80%...how's that error rate though?
    elif 0.90 > col_node["accuracy"] > 0.80:
        lg("Column=" + str(col) + " accuracy=" + str(col_node["accuracy"]) + " mse=" + str(col_node["mse"]) + " num_predictions=" + str(len(col_node["date_predictions_df"].index)))
    else:
        boom("Column=" + str(col) + " is not very accurate: accuracy=" + str(col_node["accuracy"]) + " mse=" + str(col_node["mse"]) + " num_predictions=" + str(len(col_node["predictions_df"].index)))
    # end of header line
    
    # show the timeseries forecast
    ipyDisplay(date_predictions_df)
    
    lg("")
# end of showing prediction results

Getting analysis for job=558 url=https://redten.io/ml/analysis/558/
SUCCESS - GET Analysis Response Status=200 Reason=OK
Found Job=558 analysis

--------------------------------------------------
Column=FOpen_10 accuracy=0.99848923125 mse=1.87511875363 num_predictions=50


,COpen,Date,FOpen
0,236.64,2017-02-27,237.073318
1,236.67,2017-02-28,236.157669
2,238.39,2017-03-01,237.216782
3,239.56,2017-03-02,235.210175
4,238.17,2017-03-03,233.699661
5,237.50,2017-03-06,237.479095
6,237.71,2017-03-07,235.920334
7,237.34,2017-03-08,237.982605
8,236.70,2017-03-09,237.568695
9,237.97,2017-03-10,235.905457



--------------------------------------------------
Column=FLow_5 accuracy=0.998086039927 mse=1.85004769663 num_predictions=55


,CLow,Date,FLow
0,236.35,2017-02-27,236.929581
1,236.02,2017-02-28,236.010742
2,238.37,2017-03-01,233.294571
3,238.21,2017-03-02,232.308624
4,237.73,2017-03-03,234.968628
5,237.01,2017-03-06,236.273605
6,237.71,2017-03-07,236.508759
7,236.40,2017-03-08,233.406143
8,235.74,2017-03-09,235.658737
9,236.59,2017-03-10,231.818314



--------------------------------------------------
Column=FOpen_15 accuracy=0.998673216637 mse=1.36712346617 num_predictions=45


...

,CClose,Date,FClose
0,237.11,2017-02-27,236.719559
1,236.47,2017-02-28,235.765656
2,239.78,2017-03-01,236.341888
3,238.27,2017-03-02,234.455933
4,238.42,2017-03-03,234.006851
5,237.71,2017-03-06,239.427414
6,237.71,2017-03-07,237.065964
7,236.56,2017-03-08,238.872650
8,236.86,2017-03-09,232.399292
9,237.69,2017-03-10,238.442032



--------------------------------------------------
Column=FHigh_5 accuracy=0.998197941367 mse=1.80702890975 num_predictions=55


,CHigh,Date,FHigh
0,237.31,2017-02-27,237.312714
1,236.95,2017-02-28,236.647583
2,240.32,2017-03-01,235.408691
3,239.57,2017-03-02,235.412170
4,238.61,2017-03-03,236.592255
5,238.12,2017-03-06,236.942337
6,237.71,2017-03-07,237.805847
7,237.64,2017-03-08,235.711090
8,237.24,2017-03-09,238.066299
9,238.02,2017-03-10,234.807449



--------------------------------------------------
Column=FClose_5 accuracy=0.998150179646 mse=1.98280524804 num_predictions=55


,CClose,Date,FClose
0,237.11,2017-02-27,236.434555
1,236.47,2017-02-28,236.510025
2,239.78,2017-03-01,234.041580
3,238.27,2017-03-02,232.594925
4,238.42,2017-03-03,235.686096
5,237.71,2017-03-06,236.494537
6,237.71,2017-03-07,235.777527
7,236.56,2017-03-08,234.728180
8,236.86,2017-03-09,237.905029
9,237.69,2017-03-10,231.179260


## Get the Analysis Images

In [6]:
job_res = get_job_analysis(job_id, show_plots=True)

Getting analysis for job=558 url=https://redten.io/ml/analysis/558/
SUCCESS - GET Analysis Response Status=200 Reason=OK
Found Job=558 analysis
SPY-2-558 5-Days - Predictive Accuracy
Predicted Close 5 Days vs Actual Close 5 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12122_7c49dccd964548fc.png


---------------------------------------------------------------------------------------
SPY-2-558 5-Days - Predictive Accuracy
Predicted High 5 Days vs Actual High 5 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12121_5ae84fdafa034174.png


---------------------------------------------------------------------------------------
SPY-2-558 10-Days - Predictive Accuracy
Predicted Close 10 Days vs Actual Close 10 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12120_2d87f32772344921.png


---------------------------------------------------------------------------------------
SPY-2-558 15-Days - Predictive Accuracy
Predicted Close 15 Days vs Actual Close 15 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12119_c54a1003037d4eb3.png


---------------------------------------------------------------------------------------
SPY-2-558 30-Days - Predictive Accuracy
Predicted Close 30 Days vs Actual Close 30 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12118_192e7a3c4ba840fc.png


...

---------------------------------------------------------------------------------------
SPY-2-558 30-Days - Predictive Accuracy
Predicted Open 30 Days vs Actual Open 30 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12097_214858c79e704f4c.png


---------------------------------------------------------------------------------------
SPY-2-558 25-Days - Predictive Accuracy
Predicted High 25 Days vs Actual High 25 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12096_168bcf41ff5142ad.png


---------------------------------------------------------------------------------------
SPY-2-558 15-Days - Predictive Accuracy
Predicted Open 15 Days vs Actual Open 15 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12095_de06fd5548504b41.png


---------------------------------------------------------------------------------------
SPY-2-558 5-Days - Predictive Accuracy
Predicted Low 5 Days vs Actual Low 5 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12094_354bb5267c694ba5.png


---------------------------------------------------------------------------------------
SPY-2-558 10-Days - Predictive Accuracy
Predicted Open 10 Days vs Actual Open 10 Days
URL: https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12093_e2ecdc03bc8a4f30.png


## Get the Recent Machine Learning Jobs

In [7]:
user_token = user_login(rt_user, rt_pass, rt_url)
auth_headers = {
                "Authorization" : "JWT " + str(user_token)
            }
resource_url = rt_url + "/ml/run/"
query_params = {}
post_data = {}

# Get the ML Job
resource_url = rt_url + "/ml/jobs/"

lg("Running Get ML Job url=" + str(resource_url), 6)
get_response = requests.get(resource_url, params=query_params, data=post_data, headers=auth_headers)

if get_response.status_code != 201 and get_response.status_code != 200:
    lg("Failed with GET Response Status=" + str(get_response.status_code) + " Reason=" + str(get_response.reason), 0)
    lg("Details:\n" + str(get_response.text) + "\n", 0)
else:
    lg("SUCCESS - GET Response Status=" + str(get_response.status_code) + " Reason=" + str(get_response.reason)[0:10], 5)

    as_json = True
    record = {}
    if as_json:
        record = json.loads(get_response.text)
        lg(ppj(record))
# end of post for running an ML Job

Running Get ML Job url=https://redten.io/ml/jobs/
SUCCESS - GET Response Status=200 Reason=OK
{
    "jobs": [
        {
            "algo_name": "xgb-regressor",
            "control_state": "active",
            "created": "2017-05-26 08-02-13",
            "csv_file": "",
            "desc": "Forecast simulation - 2017-05-26 08:02:13",
            "ds_name": "SPY",
            "feature_column_names": [
                "FHigh",
                "FLow",
                "FOpen",
                "FClose",
                "FVolume"
            ],
            "id": 558,
            "ignore_features": [
                "Ticker",
                "Date",
                "FDate",
                "FPrice",
                "DcsnDate",
                "Decision"
            ],
            "images": [
                "https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12122_7c49dccd964548fc.png",
                "https://rt-media.s3.amazonaws.com/media/imagesml/20170526/2_558_12121_5ae

## Redis Machine Learning Manifest

Jobs use a manifest to prevent concurrent jobs in-flight and models from colliding between users and historical machine learning jobs

![Redis is awesome](http://i.imgur.com/BobKxGp.png)

#### A manifest contains:

1. A dictionary of Redis model locations
1. S3 archival locations
1. Tracking data for import and export across environments
1. Decoupled large model files (8gb files in S3) from the tracking and deployment


In [8]:
job_manifest = get_job_cache_manifest(job_report)
lg(ppj(job_manifest))

{
    "11811": {
        "rloc": "MODELS:_MD_SPY-2-558_cd1d9a_0",
        "sloc": "redten-models-west:rt_models_userid_2_job_558_train_450_jobresults_413_modelid_11811_modelkey_MD_SPY-2-558_cd1d9a_0.cache.pickle.zlib",
        "target": "FOpen_10",
        "tracking_id": "ML_SPY-2-558_95aa375d14684a8182fe20b566090a3",
        "tracking_name": "SPY-2-558",
        "tracking_type": "UseTargetColAndUnits"
    },
    "11812": {
        "rloc": "MODELS:_MD_SPY-2-558_cd1d9a_1",
        "sloc": "redten-models-west:rt_models_userid_2_job_558_train_450_jobresults_413_modelid_11812_modelkey_MD_SPY-2-558_cd1d9a_1.cache.pickle.zlib",
        "target": "FLow_5",
        "tracking_id": "ML_SPY-2-558_95aa375d14684a8182fe20b566090a3",
        "tracking_name": "SPY-2-558",
        "tracking_type": "UseTargetColAndUnits"
    },
    "11813": {
        "rloc": "MODELS:_MD_SPY-2-558_cd1d9a_2",
        "sloc": "redten-models-west:rt_models_userid_2_job_558_train_450_jobresults_413_modelid_11813_modelkey_MD_

## Multiple Models stored in Redis

Here's how models are stored in the Redis machine learning data store

![Forecast storing multiple models in Redis](http://i.imgur.com/zUxspVL.gif)

### Conclusion

Today's talk focused on:

1. Using Redis as a machine learning data store for housing 1000s of pre-trained models
1. Streamlining model pipelines to automate build + train + predict + export/import using a REST API
1. How time intensive and expensive it is to continually rebuild machine learning models from scratch
1. The importance of tracking model accuracy and performance over time
1. How using a system like Red10 can enable an organization or team of data scientists to quickly test datasets and new ideas without stomping on each other's work
1. How this approach can make predictions from __any dataset__...not just stocks
1. This can make predictions with lots of different technologies

![Red10 Use Cases for Machine Learning](http://i.imgur.com/WWDYzGb.jpg)